In [65]:
import pandas as pd
import numpy as np
import re
from datetime import datetime


In [ ]:
def clean_price(price):
    if pd.isna(price) or price == 'N/A':
        return np.nan
    value = re.sub(r'[^\d]', '', str(price))
    return float(value) if value else np.nan

In [ ]:
def clean_area(area):
    if pd.isna(area) or area == 'N/A':
        return np.nan
    match = re.search(r'(\d+)', str(area))
    return float(match.group(1)) if match else np.nan

In [ ]:
def clean_count_values(value):
    if pd.isna(value) or value == 'N/A' or value == 'Não Tem':
        return np.nan
    if '5 Ou Mais' in str(value):
        return 5
    match = re.search(r'(\d+)', str(value))
    return int(match.group(1)) if match else np.nan


In [ ]:
def clean_date(date_str):
    if pd.isna(date_str):
        return np.nan
    return pd.to_datetime(date_str).strftime('%Y-%m-%d')

In [ ]:
def categorize_price(price):
    if pd.isna(price):
        return 'Não informado'
    elif price < 300000:
        return 'Econômico'
    elif price < 750000:
        return 'Médio'
    elif price < 1500000:
        return 'Alto padrão'
    else:
        return 'Luxo'

In [ ]:
def clean_data(df):
    # Create a copy to avoid modifying the original dataframe
    df_clean = df.copy()
    
    # Remove unnecessary columns
    columns_to_drop = ['url', 'source_file']
    df_clean.drop(columns=columns_to_drop, inplace=True)
    
    # Clean price column
    df_clean['price'] = df_clean['price'].apply(clean_price)
    
    # Clean area column
    df_clean['area_util'] = df_clean['area_util'].apply(clean_area)
    
    # Clean count columns
    df_clean['quartos'] = df_clean['quartos'].apply(clean_count_values)
    df_clean['banheiros'] = df_clean['banheiros'].apply(clean_count_values)
    df_clean['vagas'] = df_clean['vagas'].apply(clean_count_values)
    
    # Clean date column (remove time)
    df_clean['scraped_date'] = df_clean['scraped_date'].apply(clean_date)
    
    # Add derived columns
    df_clean['price_per_m2'] = df_clean.apply(
        lambda x: x['price'] / x['area_util'] if pd.notna(x['price']) and pd.notna(x['area_util']) and x['area_util'] > 0 
        else np.nan, axis=1)
    
    # Add price category
    df_clean['price_category'] = df_clean['price'].apply(categorize_price)
    
    # Reorder columns
    column_order = [
        'price', 'price_category', 'price_per_m2', 'area_util',
        'quartos', 'banheiros', 'vagas',
        'neighborhood', 'city', 'state', 'cep',
        'scraped_date'
    ]
    df_clean = df_clean[column_order]
    
    return df_clean

In [72]:
df = pd.read_csv(r'C:\Users\berna\etl_project\cleaned_location_data.csv')
df_cleaned = clean_data(df)

In [74]:
df_cleaned.nunique()

price              5941
price_category        5
price_per_m2      19678
area_util          1077
quartos               4
banheiros             5
vagas                 5
neighborhood       5089
city                491
state                27
cep               27949
scraped_date          1
dtype: int64

In [75]:
df_cleaned.to_csv('properties_cleaned_final.csv', index=False)
